<a href="https://colab.research.google.com/github/smmurdock/Regional-Sales-Analysis/blob/main/Regional_Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project issued by: Codecademy, Data Analytics Bootcamp
Project completed by: Shanay Murdock